In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

df = pd.read_csv('/content/CardioGoodFitness.csv')
print(df.shape)
df.head(5)


(180, 9)


,Product,Age,Gender,Education,MaritalStatus,Usage,Fitness,Income,Miles
0,TM195,18,Male,14,Single,3,4,29562,112
1,TM195,19,Male,15,Single,2,3,31836,75
2,TM195,19,Female,14,Partnered,4,3,30699,66
3,TM195,19,Male,12,Single,3,3,32973,85
4,TM195,20,Male,13,Partnered,4,2,35247,47


In [ ]:
# Regredir Income
df1 = df.copy()

df1['single'] = df1.apply(lambda row: 1 if row['MaritalStatus'] == 'Single' else 0, axis=1)

x = df1[['Age','Education','Usage','Fitness','Income', 'single']]
y = df1[['Miles']] # TENHO QUE USAR UM REGRESSOR!!!!!

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = LinearRegression()
model.fit(x_train, y_train)

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

print(r2_score(y_train, y_train_pred).round(0))
print(r2_score(y_test, y_test_pred).round(0))

0.7996838841960371
0.5748024987939513


In [ ]:
# Rede neural multi camada como regressor

ss = StandardScaler()
x_train_scaled = ss.fit_transform(x_train)
x_test_scaled = ss.transform(x_test)

nn = MLPRegressor(max_iter=1000)
nn.fit(x_train_scaled, y_train.values.ravel())

y_test_pred_nn = nn.predict(x_test_scaled)
y_train_pred_nn = nn.predict(x_train_scaled)

print(r2_score(y_train, y_train_pred_nn))
print(r2_score(y_test, y_test_pred_nn))


0.6035848744290655
0.3724693799378801


In [ ]:
from warnings import simplefilter
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

# optimizing
parameters = {
  'hidden_layer_sizes': [(5,), (10,), (10, 5,), (20, 10,)],
  'activation': ['relu', 'logistic', 'tanh'],
  'solver': ['adam', 'lbfgs'],
  'learning_rate': ['constant', 'adaptive'],
}
clf = GridSearchCV(nn, parameters, refit=True, return_train_score=True)
clf.fit(x_train_scaled, y_train.values.ravel())

print("Melhores parâmetros:")
print(clf.best_params_)

nn_optimized = clf.best_estimator_

y_test_pred_nn = nn_optimized.predict(x_test_scaled)
y_train_pred_nn = nn_optimized.predict(x_train_scaled)

print(r2_score(y_train, y_train_pred_nn))
print(r2_score(y_test, y_test_pred_nn))

Melhores parâmetros:
{'activation': 'relu', 'hidden_layer_sizes': (5,), 'learning_rate': 'adaptive', 'solver': 'lbfgs'}
0.8636133490074436
0.5432146602124994
